In [ ]:
import pandas as pd

In [ ]:
path = "2015_24_nifty50_15.csv"
df = pd.read_csv(path)
df.drop(columns=['Unnamed: 0','Volume'], inplace=True)

In [ ]:
df = df.tail(500).reset_index()

In [ ]:
df

# EMA 5-15 

In [41]:
from ta.trend import EMAIndicator
import numpy as np
import pandas_ta as pdta
df['Candle'] = df.apply(lambda row: 'Green' if row['Close'] >= row['Open'] else 'Red', axis=1)
df['5EMA'] = EMAIndicator(close=df['Close'], window=5, fillna=False).ema_indicator()
df['15EMA'] = EMAIndicator(close=df['Close'], window=15, fillna=False).ema_indicator()
df['CandleBody'] = abs(df['Close'] - df['Open'])

In [57]:
ce_5ema = (df['High'].shift(1) < df['15EMA'].shift(1)) & (df['High'] >= df['15EMA'])
pe_5ema = (df['Low'].shift(1) > df['5EMA'].shift(1)) &(df['Low'] <= df['5EMA'])

df['Signal_5EMA'] = np.select([ce_5ema, pe_5ema],['CE', 'PE'], default='None')

In [58]:
df[df['Signal_5EMA']!= "None"]

,index,Datetime,Open,High,Low,Close,5EMA,15EMA,Candle,CandleBody,Signal_5EMA,Super_Trend
6,39698,2023-12-15 10:45:00,21316.90,21317.90,21281.80,21312.70,21309.407819,NaN,Red,4.20,PE,1
25,39717,2023-12-18 09:15:00,21434.80,21471.60,21365.35,21462.10,21429.462938,21377.477402,Green,27.30,PE,1
35,39727,2023-12-18 11:45:00,21476.90,21476.95,21448.35,21449.60,21462.512280,21439.514076,Red,27.30,PE,1
41,39733,2023-12-18 13:15:00,21416.80,21439.75,21415.30,21438.50,21436.683520,21437.824753,Green,21.70,CE,1
44,39736,2023-12-18 14:00:00,21423.50,21443.35,21422.05,21434.80,21431.246969,21434.543145,Green,11.30,CE,1
...,...,...,...,...,...,...,...,...,...,...,...,...
453,40145,2024-01-11 10:00:00,21674.70,21675.15,21654.70,21672.20,21661.778185,21609.438238,Red,2.50,PE,1
486,40178,2024-01-12 12:00:00,21835.10,21836.15,21810.65,21827.55,21828.272553,21785.126322,Red,7.55,PE,1
491,40183,2024-01-12 13:15:00,21842.95,21864.25,21841.20,21856.75,21844.111406,21813.297384,Green,13.80,PE,1
494,40186,2024-01-12 14:00:00,21894.60,21896.45,21864.10,21879.10,21873.003380,21837.299224,Red,15.50,PE,1


In [ ]:
def EMA_5_15():
    df['Candle'] = df.apply(lambda row: 'Green' if row['Close'] >= row['Open'] else 'Red', axis=1)
    df['5EMA'] = EMAIndicator(close=df['Close'], window=5, fillna=False).ema_indicator()
    df['15EMA'] = EMAIndicator(close=df['Close'], window=15, fillna=False).ema_indicator()
    df['CandleBody'] = abs(df['Close'] - df['Open'])
    ce_5ema = (df['High'].shift(1) < df['15EMA'].shift(1)) & (df['High'] >= df['15EMA'])
    pe_5ema = (df['Low'].shift(1) > df['5EMA'].shift(1)) &(df['Low'] <= df['5EMA'])
    df['Signal_5EMA'] = np.select([ce_5ema, pe_5ema],['CE', 'PE'], default='None')
    

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime,timedelta
import time

symbol = "^NSEI"
start_date = datetime(2024,1,30,9,15)#'2024-01-29'
end_date = datetime(2024,2,2,3,30) #'2024-02-02'
IsPreviousFrameTaken = False

class CNSEData:
    def __init__(self):
        self._min_data = pd.DataFrame(columns=['High', 'Low', 'Open', 'Close'])
        self._hour_data = pd.DataFrame(columns=['High', 'Low', 'Open', 'Close'])

    # Getters and setters for 1min candle data
    def get_min_high(self):
        return float(self._min_data['High'][1:].iloc[0])

    def get_min_low(self):
        return float(self._min_data['Low'][1:].iloc[0])

    def get_min_open(self):
        return float(self._min_data['Open'][1:].iloc[0])

    def get_min_close(self):
        print(self._min_data.iloc[0])
        return float(self._min_data['Close'].iloc[0])

    def set_min_data(self, min_data):
        self._min_data = min_data

    # Getters and setters for 1hr candle data
    def get_hour_high(self):
        return float(self._hour_data['High'][1:].iloc[0])

    def get_hour_low(self):
        return float(self._hour_data['Low'][1:].iloc[0])

    def get_hour_open(self):
        return float(self._hour_data['Open'][1:].iloc[0])

    def get_hour_close(self):
        # print(self._hour_data.iloc[0])
        return float(self._hour_data['Close'].iloc[0])

    def set_hour_data(self, hour_data):
        self._hour_data = hour_data

    # Method to initialize data
    def initialize_1min_data(self):
        self._min_data = yf.download(symbol, start=start_date, end=end_date, interval='1m')
        self._min_data.to_excel("nifty50_data_1min_last_week.xlsx", index=False)

    # Method to initialize data
    def initialize_1hr_data(self):
        self._hour_data = yf.download(symbol, start=start_date, end=end_date, interval='1h')
        self._hour_data.to_excel("nifty50_data_1hr_last_week.xlsx", index=False)

    # Method to update 1min candle data frame
    def update_1min_data_frame(self):
        self._min_data = self._min_data.iloc[1:]  # Delete first candle
        # Update every private member
        
    # Method to update 1hr candle data frame
    def update_1hr_data_frame(self):
        self._hour_data = self._hour_data.iloc[1:]  # Delete first 1hr candle
        # Update every private member

class CMAINLOGIC:
    def __init__(self):
        self.index = -7
        self._hour_data = pd.DataFrame(columns=['High', 'Low', 'Open', 'Close'])

    # Method to initialize data
    def initialize_1hr_data(self):
        self._hour_data = yf.download(symbol, start=start_date - timedelta(days=7), end=start_date, interval='1h')
        self._hour_data.to_excel("nifty50_data_1hr_previous_last_week.xlsx", index=False)

    def IsMySetupReady(self):
        self.isSetupReady = self.Setup()
        return self.isSetupReady

    # Method to update 1hr candle data frame
    def update_1hr_data_frame(self):
        self.index =  self.index + 1

    def get_previous_hour_low(self):
        return float(self._hour_data['Low'].iloc[self.index])
        
    def get_hour_close(self):
        print(self._hour_data.iloc[self.index])
        return float(self._hour_data['Close'].iloc[self.index])
    
    def Setup(self):
        prev_candle_high = self._hour_data['High'][self.index]
        prev_18_candle_low = self._hour_data['Low'][self.index - 18 : self.index].min()
        print(f"prev_18_candle_low: {prev_18_candle_low}")
        if (prev_candle_high - prev_18_candle_low > 100):
            return True
        return False


if __name__ == "__main__":
    cnse_data = CNSEData()
    cmain_logic = CMAINLOGIC()

    cnse_data.initialize_1min_data()
    cnse_data.initialize_1hr_data()
    
    cmain_logic.initialize_1hr_data()
    iCount=0
    m_iCurrentPrice = 0
    m_iprevious_1hr_low = 0
    bEntry = False
    m_iTargetValue = 0
    m_iStoploss = 0
    while True:  
        m_iCurrentPrice = cnse_data.get_min_close()
        m_iprevious_1hr_low = cmain_logic.get_previous_hour_low()

        if(iCount==60):
            # print("Initial 1-hr  Close:", cnse_data.get_hour_close())
            # print("Initial 1-hr  Close:", cmain_logic.get_hour_close())
            iCount = 0
            cmain_logic.update_1hr_data_frame()

        if(bEntry == False):
            if(cmain_logic.IsMySetupReady()):
                if(m_iCurrentPrice < m_iprevious_1hr_low):
                    bEntry = True
                    m_iTargetValue = m_iCurrentPrice - 100
                    m_iStoploss = m_iCurrentPrice + 25
                    print(f"Entry : {m_iCurrentPrice} 1hr_low : {m_iprevious_1hr_low}")
        
        if(bEntry == True):
            if(m_iCurrentPrice < m_iTargetValue):
                print(f"Target Hit : {m_iCurrentPrice}")
                m_iCurrentPrice = 0
                m_iprevious_1hr_low = 0
                bEntry = False
                m_iTargetValue = 0
                m_iStoploss = 0
                bEntry = False
                break
            if(m_iCurrentPrice > m_iStoploss):
                print(f"stoploss hit : {m_iCurrentPrice}")
                bEntry = False
                m_iCurrentPrice = 0
                m_iprevious_1hr_low = 0
                bEntry = False
                m_iTargetValue = 0
                m_iStoploss = 0
                break

        cnse_data.update_1min_data_frame()
        iCount=iCount+1
        # time.sleep(3)

Exception in thread Thread-5 (_run_via_pool):
Traceback (most recent call last):
  File "C:\Python\Lib\site-packages\requests\models.py", line 971, in json
    return complexjson.loads(self.text, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Python\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Python\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Python\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Python\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Python\Lib\threading.py", line 975, 

In [ ]:
# def Test1():
#     return "HI"

# def case_example(argument):
#     cases = {
#         "case1": Test1(),
#         "case2": "This is Case 2",
#         "case3": "This is Case 3",
#         "default": "This is the default case"
#     }
    
#     return cases.get(argument, cases["default"])

# print(case_example("case1"))
